## Time Series Exercises for Ay 119

<b>Written by:</b> Matthew J. Graham (Caltech), May 2020

<b>Dependencies:</b>

   * numpy
   * pandas
   * astropy.timeseries
   * sklearn.datasets
   * GPy (pip install GPy)


### Period finding

Since period finding is one of the main time series analysis techniques in astronomy, we're going to explore some of the dependencies of the most popular period finding algorithm -- the Lomb-Scargle periodogram (see Jake Vanderplas' excellent review article on this: https://arxiv.org/abs/1703.09824 and with associated code at https://github.com/jakevdp/PracticalLombScargle/). In particular, we want to assess its performance as a function of signal-to-noise, time series sampling, and waveform shape. It is often worth investigating the performance of algorithms in terms of toy data sets to get an understanding of what the limitations may be. So:

1) Write a routine to generate a periodic time series of a function $perfunc$ at a period of $per$, containing $n$ datapoints, assuming homoscedastic Gaussian errors given by a variance $sigma^2$, with a mean sample time of $meandt$, and a flag to indicate whether the sampling is regular or irregular:



In [96]:
import math
import pandas as pd
import numpy as np

eps = 1e-12

def getTimeSeries(perfunc, per, n, sigma2, meandt, regular_sample = True):
        ts = [0] * n
        t = 0
        for i in range(n):
            val = perfunc(t, per)
            if val > eps:
                ts[i] = perfunc(t, per)
            t += 1
        return ts

Demonstrate it with (a) a sinusoidal waveform ($y(t) = \sin(2 \pi t / per)$) and (b) a square waveform and apply LombScargle to recover the period (so choose a range of test frequencies, or use autopower (see the documentation for the astropy.timeseries.LombScargle method) and then plot the corresponding periodogram, and the phase folded time series ($phi = t / per - int(t / per)$. 

In [21]:
demo_wf1 = lambda t, per: math.sin(2*math.pi*t/per)

ts1 = getTimeSeries(demo_wf1, math.pi, 5, 0, 0, regular_sample=True)

print(ts1)

[0, 0.9092974268256817, 0, 0, 0.9893582466233818]


2) For a range of periods extending over three decades, e.g., $\log_{10}(per) = 1 - 3$, generate 100 time series and determine how accurate (how many period found by Lomb-Scargle are within 1\% of the known periods) Lomb-Scargle is as:

a) a function of number of data points in the time series, i.e., plot LS accuracy againts $n$

b) a range of error values ($sigma^2$)

c) different sample times, both regular and irregular ($dt = meandt + random value$)

Does Lomb-Scargle do better for a sinusoidal waveform than the square waveform?

### Gaussian processes

A Gaussian process is an optimal way (in a Bayesian sense) to fit a time series and can be used to predict (interpolate) values where needed as well as forecasting (extrapolating). The standard set we are going to look at consists of monthly average $CO_2$ concentrations collected at the Mauna Loa Observatory in Hawaii between 1958 and 2001. We want to model the $CO_2$ concentration as a function of time.

The data is available as a standard data set:

In [46]:
from sklearn.datasets import fetch_openml
# mauna-loa-atmospheric-co2
# co2_data = fetch_openml(data_id=41187)
# X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X, y = fetch_openml('mauna-loa-atmospheric-co2', version=1, return_X_y=True, as_frame=False)

In [58]:
# features are: listed here https://www.openml.org/d/41187
# X.shape
# y.shape
# print(X)
COL_NAMES = ['co2', 'year', 'month', 'day', 'weight', 'flag', 'station']

feats = pd.DataFrame(X, columns=COL_NAMES[1:])
target = pd.DataFrame(y, columns=COL_NAMES[:1])

In [60]:
feats

,year,month,day,weight,flag,station
0,1958.0,3.0,29.0,4.0,0.0,0.0
1,1958.0,4.0,5.0,6.0,0.0,0.0
2,1958.0,4.0,12.0,4.0,0.0,0.0
3,1958.0,4.0,19.0,6.0,0.0,0.0
4,1958.0,4.0,26.0,2.0,0.0,0.0
...,...,...,...,...,...,...
2220,2001.0,12.0,1.0,7.0,0.0,0.0
2221,2001.0,12.0,8.0,7.0,0.0,0.0
2222,2001.0,12.0,15.0,7.0,0.0,0.0
2223,2001.0,12.0,22.0,6.0,0.0,0.0


In [121]:
target

,co2
0,316.1
1,317.3
2,317.6
3,317.5
4,316.4
...,...
2220,370.3
2221,370.8
2222,371.2
2223,371.3


In [62]:
target.mean()

co2    340.142247
dtype: float64

In [64]:
t = feats['year'].to_numpy()
print(t)

[1958. 1958. 1958. ... 2001. 2001. 2001.]


A bit of preprocessing is required to convert this to average monthly counts. Plot the data.

Now we are going to use the GPy library to fit our time series $(t, y)$ and the way to do this is:

In [65]:
import GPy
kern = GPy.kern.RBF(1) # define the kernel here
yp = y - y.mean() # subtract mean from observed value as we're assuming a zero mean process
m = GPy.models.GPRegression(t[:, None], yp[:, None], kern) # define GP regressor
m.optimize() # fit

In [75]:
tp = t - t.mean()
print(tp[:, None])

[[-22.02606742]
 [-22.02606742]
 [-22.02606742]
 ...
 [ 20.97393258]
 [ 20.97393258]
 [ 20.97393258]]


This performs a maximum likelihood estimation of the Gaussian process kernel hyperparameters 
and we can then predict values:

In [143]:
# tpred = np.linspace(2002, 2021, 20)
tpred = np.linspace(1980,1999,20)
# tpredp = tpred - np.concatenate((t,tpred)).mean()
ypred = m.predict(tpred[:, None]) # This will return the predicted values at tpred and the predicted error

In [144]:
ypred

(array([[-1.76338534],
        [-0.34172268],
        [ 1.05667544],
        [ 2.48318874],
        [ 3.99412721],
        [ 5.62263987],
        [ 7.35795557],
        [ 9.14165572],
        [10.88412935],
        [12.49616256],
        [13.92411281],
        [15.17501563],
        [16.32118942],
        [17.48120877],
        [18.78265949],
        [20.31860407],
        [22.11181615],
        [24.09801255],
        [26.1328355 ],
        [28.01975596]]), array([[3.6524959 ],
        [3.65254252],
        [3.6526398 ],
        [3.65274898],
        [3.65282716],
        [3.65285345],
        [3.65281614],
        [3.65271888],
        [3.65263937],
        [3.65271219],
        [3.65294575],
        [3.65312654],
        [3.6531165 ],
        [3.65319987],
        [3.65372963],
        [3.65436536],
        [3.65449195],
        [3.65493313],
        [3.65744029],
        [3.66009   ]]))

So we want to model the time series as a combination of:

* a long term, smooth rising trend (using an RBF kernel)
* a seasonal component with a fixed periodicity of 1 year (using a PeriodicExponential)
* smaller, medium term irregularities (using a RationalQuadratic (RatQuad) kernel)
* a noise term (consisting of a RBF kernel and a White kernel)

Define the component kernels: 

In [ ]:
k1 = GPy.kern....

And then define the complete kernel:

In [ ]:
k = k1 + ...

Now fit it to the data, and then plot the model against the measured data. Extend the plot to 2030 and see what the mode suggests the trend is.